# Trajectories
--------------------

As a first look at the code, this notebook shows the main procedures and features to deal with trajectories: reading, manipulating, customizing.

## Opening and handling a trajectory
----------------------------------------------

A trajectory file can directly be read using the `Trajectory` class. It only requires a path to the file, and some additional arguments (`fmt`, `backend`, `first`, `last`, `step`) that will be detailed later.

Let's open our first trajectory file:

In [1]:
from pysc import Trajectory

traj = Trajectory('../data/kalj_N150.xyz')

A `Trajectory` object is iterable, and is composed of `System` objects (*i.e.* frames).

In [2]:
# Number of frames in the trajectory
print(len(traj))

101


The `first`, `last` and `step` parameters allow to slice the trajectory to select a subset of the frames:
* `first` is the index of the first frame to read (default is `0`) ;
* `last` is the index of the last frame to read (default is `None`, meaning it will read up to the last frame) ;
* `step` is the increment between two consecutive frames to read (default is `1`, meaning that every frame is read). Setting `step=2` means that every 2 frames only will be read ;

Let's have a look at the first frame:

In [3]:
print(traj[0])

System(number_of_particles=150, species=['A' 'B'], chemical_fractions=[0.8 0.2], cell=[5. 5. 5.])


This tells us about the main features of the system. In particular, a `System` is composed of a cell (*i.e.* simulation box) that must be **rectangular**.

In [4]:
print(traj[0].cell)

Cell(side=[5. 5. 5.], volume=125.0)


The code accepts bidimensional and tridimensional systems. The spatial dimension is guessed by the number of sides the cell has in the input file.

A `System` is also composed of particles, accessible via the iterable`particle` attribute:

In [5]:
print(traj[0].particle[0])

Particle(position=[ 0.819926 -1.760086  0.36845 ], species=A, label=-1, index=140600789906488, species_id=1)


Finally, a particle has several attributes: 
- a position ; 
- a type, or *species* ;
- a label, which is the index of the cluster that particle belongs to. Its default value is `-1` but it is automatically assigned after a clustering ;
- an index, which is a unique integer value to identify the particle ;

In [6]:
p = traj[0].particle[0]

print('Position :', p.position)
print('Species :', p.species)
print('Label :', p.label)
print('Index :', p.index)

Position : [ 0.819926 -1.760086  0.36845 ]
Species : A
Label : -1
Index : 140600789906488


An arbitrary property can be given to a particle, either by the user or when reading a trajectory file that contains additional properties.

In [7]:
p.mass = 0.5
print(p)

Particle(position=[ 0.819926 -1.760086  0.36845 ], species=A, label=-1, index=140600789906488, species_id=1, mass=0.5)


All these properties can be accessed via the method `dump`, either from a `System` or a `Trajectory`. This will return a numpy array:

In [8]:
# Positions for the first frame and the 5 first particles
pos_0 = traj[0].dump('position')
print(pos_0[0:5])

[[ 0.819926 -1.760086  0.36845 ]
 [ 0.29098   1.24579  -2.495344]
 [ 2.484607  0.35245   1.371003]
 [-1.02062  -1.927986  0.663132]
 [-0.816276  2.062004  0.421107]]


This method accepts aliases for common particle properties, such as the position of the species:

In [9]:
pos_0 = traj[0].dump('pos')
spe_0 = traj[0].dump('spe')

print(pos_0[0:5])
print(spe_0[0:5])

[[ 0.819926 -1.760086  0.36845 ]
 [ 0.29098   1.24579  -2.495344]
 [ 2.484607  0.35245   1.371003]
 [-1.02062  -1.927986  0.663132]
 [-0.816276  2.062004  0.421107]]
['A' 'A' 'A' 'A' 'A']


However, any other additional particle property should be accessed using the following syntax:

`dump('particle.property')`

where `property` is the name of the corresponding particle property, e.g. `particle.mass`, `particle.radius`, etc. Obviously, this property must be defined for **all** the particles (not the case here) for the  `dump` method to work.

Finally, some properties of our `System`:

In [10]:
sys = traj[0]

print('Number of spatial dimensions :', sys.number_of_dimensions)
print('Number of particles :', len(sys.particle))
print('Density :', sys.density)
print('Distinct species :', sys.distinct_species)
print('Chemical fractions :', sys.chemical_fractions)
print('Pairs of species :', sys.pairs_of_species)
print('Pairs of species ID :', sys.pairs_of_species_id)

Number of spatial dimensions : 3
Number of particles : 150
Density : 1.2
Distinct species : ['A' 'B']
Chemical fractions : [0.8 0.2]
Pairs of species : [('A', 'A'), ('A', 'B'), ('B', 'A'), ('B', 'B')]
Pairs of species ID : [(1, 1), (1, 2), (2, 1), (2, 2)]


A special emphasis on the very last property, `pairs_of_species_id`: this is a standardization of all the species in the trajectory, from a litteral name or symbol to an integer value. Here, for example, we have:
- `"A"` --> `1`
- `"B"` --> `2`

In more complex cases, *e.g.* having atomic symbols such as `"Si"`, `"C"`, `"O"`, etc., this standardization can come in handy.

## Dealing with different trajectory formats
-----------------------------------------------------

Natively, the code only reads/write standard [XYZ](https://en.wikipedia.org/wiki/XYZ_file_format) trajectories (with a `cell` attribute in the comment line) and [RUMD](http://rumd.org/) trajectories:

In [11]:
# Open a XYZ trajectory
#  default value for `fmt` is "xyz"
traj_xyz = Trajectory('../data/kalj_N150.xyz', fmt='xyz')
print(traj_xyz)

# Open a RUMD trajectory
traj_rumd = Trajectory('../data/kalj_N256_rho1.185_rumd.xyz.gz', fmt='rumd')
print(traj_rumd)

Trajectory(filename="../data/kalj_N150.xyz", number_of_frames=101)
Trajectory(filename="../data/kalj_N256_rho1.185_rumd.xyz.gz", number_of_frames=1)


However, it is possible to rely on external packages to read more formats such as [LAMMPS](https://lammps.sandia.gov/), [GROMACS](https://www.gromacs.org/), etc.

These two external packages are:
* [atooms](https://framagit.org/atooms/atooms), a Python framework for simulations of interacting particles ;
* [MDTraj](https://www.mdtraj.org/1.9.5/index.html), a Python library that allows users to manipulate molecular dynamics trajectories ;

With these dependencies properly installed in the Python environment, it is possible to use them as backend when creating an instance of `Trajectory`:

In [12]:
# Open a LAMMPS trajectory using atooms
traj = Trajectory('../data/lj_N256_rho1.0.atom', 
                  fmt='lammps', backend='atooms')

print('Number of frames :', len(traj))
print(traj[0])

Number of frames : 6
System(number_of_particles=256, species=['1'], chemical_fractions=[1.], cell=[6.3496 6.3496 6.3496])


When using `backend="atooms"`, it is **absolutely essential** to set the `fmt` parameter to the right value for the appropriate parser to be selected, indepedent of the file extension.

When using `backend="mdtraj"`, however, the file extension is used to determine the correct format. The `fmt` parameter can thus be ignored:

In [13]:
# Open a PDB trajectory using MDTraj
traj = Trajectory('../data/frame0.pdb', backend='mdtraj')

print('Number of frames :', len(traj))
print(traj[0])

Number of frames : 501
System(number_of_particles=22, species=['C' 'H' 'N' 'O'], chemical_fractions=[0.27272727 0.54545455 0.09090909 0.09090909], cell=[1. 1. 1.])


## Dealing with additional particle properties
--------------------------------------------------------

Some trajectory formats may contain additional particle properties (mass, radius, etc.). It is possible to read these properties from the file using the parameter `additional_fields` when creating an instance of `Trajectory`.

Let's consider a simple XYZ trajectory file called `traj_with_masses.xyz` with 2 frames and only 3 particles, as follows:

`
3
columns:id,pos,mass cell:1.0,1.0,1.0
A 0.1 0.0 0.0 0.5
B 0.0 0.1 0.0 0.25
B 0.0 0.0 0.1 0.25
3
columns:id,pos,mass cell:1.0,1.0,1.0
A 0.2 0.0 0.0 0.5
B 0.0 0.2 0.0 0.25
B 0.0 0.0 0.2 0.25
`

As long as the property is specified in the header, it can be read when opening the trajectory:

In [14]:
# Specifying an additional field
traj = Trajectory('../data/traj_with_masses.xyz', 
                  additional_fields=['mass'])

print(traj[0].particle[0])
print(traj.dump('particle.mass'))

Particle(position=[0.1 0.  0. ], species=A, label=-1, index=140600074187776, mass=0.5, species_id=1)
[array([0.5 , 0.25, 0.25]), array([0.5 , 0.25, 0.25])]


**Important:**

Not all trajectory formats and backends support additional fields.

When reading, the `additional_fields` parameter may then be ignored (only positions and species will be read). The same applies when writing a trajectory.

On that regard, the XYZ format is probably the most flexible, being natively supported by the code.

## Setting or modifying a particle property
-----------------------------------------------------

The method `set_property` allows to set or change a particle property trajectory-wide, for all particles or for a specific subset. This property can be either a scalar value, in which case it will be set to all the particles in the subset:

In [15]:
traj = Trajectory('../data/traj_with_masses.xyz', 
                  additional_fields=['mass'])

# Current `mass` property
print(traj.dump('particle.mass'))

# Change it for all the particles
traj.set_property('mass', 1.0)
print(traj.dump('particle.mass'))

# Change it for a given subset
traj.set_property('mass', 0.75, subset="species == 'B'")
print(traj.dump('particle.mass'))

[array([0.5 , 0.25, 0.25]), array([0.5 , 0.25, 0.25])]
[array([1., 1., 1.]), array([1., 1., 1.])]
[array([1.  , 0.75, 0.75]), array([1.  , 0.75, 0.75])]


The property can also be a `list` or a `numpy.ndarray`. This is convenient if this property is *e.g.* read from another file or computed elsewhere in your code.

In this case, the parameter should have the right shape: same number of frames than the trajectory, and same number of particles than the considered subset:

In [16]:
# 2 frames, 3 particles (same as the trajectory)
masses = [[0.5, 0.25, 0.25],
          [0.6, 0.35, 0.35]]
traj.set_property('mass', masses)
print(traj.dump('particle.mass'))

# 2 frames, 2 particles 
#  (because there are only B particles in the subset)
masses = [[0.23, 0.34],
          [0.56, 0.67]]
traj.set_property('mass', masses, "species == 'B'")
print(traj.dump('particle.mass'))

[array([0.5 , 0.25, 0.25]), array([0.6 , 0.35, 0.35])]
[array([0.5 , 0.23, 0.34]), array([0.6 , 0.56, 0.67])]


The property can also be a **new** property that the particles do not have yet:

In [17]:
# Set a new property
radii = [[0.5, 0.4, 0.4],
         [0.5, 0.4, 0.4]]
traj.set_property('radius', radii)
print(traj.dump('particle.radius'))

[array([0.5, 0.4, 0.4]), array([0.5, 0.4, 0.4])]


As mentioned above, it is absolutely essential that **all** particles have the property when calling the `dump` method or when applying a filter on a structural descriptor (see the **notebook #3**).